# Χρησιμοποιήθηκαν 2 free pass

In [12]:
import pandas as pd
import numpy as np
import networkx as nx
import dask.dataframe as dd
import time

<span style="color:black; font-weight:700;font-size:18px"> Διαβάζουμε το αρχείο με τις τριάδες και φιλτράρουμε ανά iterate την συνθήκη που υπάρχει, πάνω από 30 reviews ανα χρήστη και άνω από 50 review ανά επιχείρηση. </span>

In [13]:
df = pd.read_csv('philly_users_businesses_stars.csv')
for i in range(11):
    df.columns
    # another way 
    df = df[df.user_id.isin(df.user_id.value_counts().loc[lambda x: x > 30].index)]
    #df = df[df.groupby('user_id')['user_id'].transform('size').gt(15)]
    print(len(df.index))
    # another way 
    df = df[df.business_id.isin(df.business_id.value_counts().loc[lambda x: x > 50].index)]
    #df = df[df['business_id'].map(df['business_id'].value_counts()).gt(15)]
    print(len(df.index))
    df = df.dropna()
    print("#####################################")

272933
167056
#####################################
132040
117066
#####################################
109447
106727
#####################################
104789
103946
#####################################
103320
103070
#####################################
102951
102852
#####################################
102672
102472
#####################################
102292
102093
#####################################
101973
101923
#####################################
101923
101923
#####################################
101923
101923
#####################################


<span style="color:black; font-weight:700;font-size:18px"> Εδώ διαβάζουμε το αρχείο με τους φίλους κάθε χρήστη. Λόγω του μεγάλου μεγέθους του αρχείου χρησιμοποιούμε το dask για να κάνουμε batches των 1000000 χρηστών και όσες γραμμές έχουν στην στήλη user_id έναν χρήστη που ανήκει στο friends_series αποθηκεύεται.</span>

In [14]:
# Load the DataFrame using dask
users = dd.read_json('yelp_academic_dataset_user.json', blocksize=1e6)
friends_series = pd.Series(df['user_id'].values)

# Define a function to filter the DataFrame based on the values in x
def filter_data(df):
    return df[df['user_id'].isin(friends_series)]

# Apply the filter_data function to each chunk of the DataFrame in parallel
result = users.map_partitions(filter_data).compute()

result.to_csv('user_friends_unfiltered.csv')

<span style="color:black; font-weight:700;font-size:18px"> Τώρα έχουμε ένα DataFrame με δύο στήλες, μια στήλη τα user_id και μια άλλη που έχει σε μορφή λίστας τους φίλους του χρήστη. </span>

In [15]:
friends_of_users = result[['user_id', 'friends']].copy()
friends_of_users['friends'] = friends_of_users['friends'].apply(lambda x: x.split(', '))
friends_of_users.head

<bound method NDFrame.head of                     user_id                                            friends
11   IpLRJY4CP3fXtlEd8Y4GFQ  [hdwDo7CLh9aN_9PckAos4Q, ci-mepWGgsgGT3sFI2mhM...
17   RgDVC3ZUBqpEe6Y1kPhIpw  [sHozd2pcOKwHtPr8VlZJfg, 7mL9cvICl8fuCQTM89a-S...
25   KrIL3TIOJI-tjvU6BlcA-g  [eUlLKiRbX3k4VS3Ko_WxTA, nmiOEav_m5eT08zz8BIi4...
1    ouODopBKF3AqfCkuQEnrDg  [tlEHfYMPIm8-AMMk3T1s0w, aOkQtJvCvGzLUNVtaGb9x...
47   Z2pXm4f-2O5ZpMUmXuYK0Q  [c4eh4aq5Bw9zVMt4xLG2ew, nBdv5Br4G8pDWXXqSXhUu...
..                      ...                                                ...
885  iUo7JE5Uj_LsNH0BxXh_kQ  [ZuRBTpKHifS0aXplqJLvGg, hv-GtL9hsBqz8oM4vteGi...
884  DrvmH6hJifrAyqQR0Sliww  [YljmfktfbakMINbP1G-gKg, gWZETLPu_ihAtLOYtrm0m...
267  TdW4MVRT2MGdFPD6DoZSbQ  [gWZETLPu_ihAtLOYtrm0mw, 6BkR1OS11VsARi1typB3_...
893  v9FT1O5vdOHlzOYpyc-Y2w                                             [None]
8    U165gjpO81auHqPTZbf3TQ  [hukAFNR0SeHLnHCBLI-TSQ, gok-GLiwpkjJvpOfaqnKR...

[1604 rows x 2 column

<span style="color:black; font-weight:700;font-size:18px"> Εδώ δημιουργούμε το γράφημα. Χρησιμοποιώντας την iterrows πηγαίνουμε από γραμμή σε γραμμή, κρατάμε τον χρήστη και δημιουργούμε ακμές με όλους τους φίλους του, μονάχα βέβαια άμα δεν έχουν μπει ήδη. Τo networkx δημιουργεί αυτόματα έναν νέο κόμβο με κάθε νέα ακμή. Στο τέλος κρατάμε την μεγαλύτερη συνεκτική συνιστώσα και βλέπουμε ότι πράγματι έχουμε 1504 χρήστες-κόμβους και 887 επιχειρήσεις.</span>

In [16]:
G = nx.Graph()
user_nodes = friends_of_users['user_id'].values 
G.add_nodes_from(user_nodes) 

for i, row in friends_of_users.iterrows():
    node = row['user_id'] # Get the node from the first column of the row
    friends = row[1] # Get the list of friends from the second column of the row
    for friend in friends:
        if(friend in user_nodes): # Check if the friend is already a node in the graph
            G.add_edge(node, friend) # Add an edge to the graph between the node and the friend
largest_cc = max(nx.connected_components(G), key=len)
CC_max = nx.subgraph(G,largest_cc)
print("Amount of nodes:",len(CC_max.nodes()))
print("Amount of edges:",len(CC_max.edges()))

Amount of nodes: 1504
Amount of edges: 19347


In [17]:
df = df[df['user_id'].isin(list(CC_max.nodes()))]
N = df.drop_duplicates(subset = ['user_id'])
print(N)
M = df.drop_duplicates(subset = ['business_id'])
print(M)

                       user_id             business_id  stars
3       smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ    4.0
5       IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg    5.0
10      ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7Aur9o4kQQ    5.0
11      IKbjLnfBQtEyVzEu8CuOLg  VJEzpfLs_Jnzgqh5A_FVTg    4.0
13      NUtIAX-ygn474tDg5nmesg  6LCZLGa09Qifn6rG7-DNrg    4.0
...                        ...                     ...    ...
113676  k6F3Ig1Sj1IqPfTgKhOkzw  cGX-1IUwXOjkUqZbkKYcjw    4.0
126492  RCZ5M9o2-fxgFuurpmEs3w  cGX-1IUwXOjkUqZbkKYcjw    4.0
130758  WIDiCSoG68ScNeLLhORN6A  w55VlTgAoRXPnqFte3j9ew    5.0
164063  g8Wrs02Za0rkbFTH7rt3lw  Ldch7Nc5gaZrhcIRd7mcjw    4.0
169949  nUD_Duu-NR7qfyq8fgqiAQ  KyDcAU41QDQpYAFyK6Q6hg    5.0

[1504 rows x 3 columns]
                       user_id             business_id  stars
3       smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ    4.0
5       IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg    5.0
10      ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7

<span style="color:black; font-weight:700;font-size:18px"> Από το Ν που έχει τους χρήστες, ανακατεύουμε και παίρνουμε τους πρώτους 100 στην μεταβλητή test_users. Από το DataFrame df παίρνουμε όσες τριάδες περιέχουν user_id από το test_users</span>

In [18]:
N = N.sample(frac=1)
test_users = N.iloc[:100]
union_bus = df[df.user_id.isin(test_users.user_id)]

<span style="color:black; font-weight:700;font-size:18px"> Βρίσκουμε τώρα ποιοι από τους χρήστες έχουν κάνει review στα ίδια εστιατόρια.</span>

In [19]:
grouped = union_bus.groupby('business_id')['user_id'].apply(set)
filtered = grouped[grouped.apply(len) > 1]
test_3 = union_bus[union_bus['business_id'].isin(filtered.index)]

<span style="color:black; font-weight:700;font-size:18px"> Τώρα έχουμε ένα DataFrame test_3 με τριάδες όπου στα εστιατόρια έχουν κάνει review καποιοι από τους 100 που επιλέχθηκαν. Ανακατεύουμε το DataFrame και από επιλέγουμε τα πρώτα 100. Τώρα από το DataFrame test_3 κρατάμε μόνο τις τριάδες που έχουν business_id μέσα από τα 100 που επιλέχθηκαν. </span>

In [20]:
M = test_3.drop_duplicates(subset = ['business_id'])
M = M.sample(frac=1)
test_bus = M.iloc[:100]
final_test = test_3[test_3.business_id.isin(test_bus.business_id)]
print(final_test)

                       user_id             business_id  stars
1202    R3L9EHSSEtdO-Kcunmzutg  QWqKTWQ2OiDgo3dzNkpung    4.0
3984    h9x6C4vKVkkS7TVTWflB4g  tzVLTQNUtiCAxUZmEuF0vQ    4.0
5659    -QmEKJ_CzZnT9biZHddfZQ  tzVLTQNUtiCAxUZmEuF0vQ    4.0
6101    Sg_BvnTHsbc1AABiJjaiPA  wUnLSg_GKfEIQ5CQQ770_g    4.0
10270   vzNYQaFyBnIU_bBkJsUJQA  QWqKTWQ2OiDgo3dzNkpung    5.0
...                        ...                     ...    ...
962227  BJYnLAu8SkwfiDogbADBtA  qISf5ojuYbD9h71NumGUQA    3.0
963146  BJYnLAu8SkwfiDogbADBtA  rgeuy1qbw6Z8B6CSVANHIA    4.0
965917  tUW0AXg4C3H9TP5lyhoQuA  c6hhh1Kpr7iau8qMH74w2Q    5.0
966514  RgDVC3ZUBqpEe6Y1kPhIpw  c6hhh1Kpr7iau8qMH74w2Q    5.0
966706  QVNFi8ssjPtHY5I9M1vb8Q  jkGQQ4_LgJx3hwPtCFkzbQ    1.0

[727 rows x 3 columns]


In [21]:
final_train = df.merge(final_test,on=['user_id','business_id'],how='left')
final_train = final_train.drop('stars_y', axis=1)
final_train.head

<bound method NDFrame.head of                       user_id             business_id  stars_x
0      smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ      4.0
1      IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg      5.0
2      ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7Aur9o4kQQ      5.0
3      IKbjLnfBQtEyVzEu8CuOLg  VJEzpfLs_Jnzgqh5A_FVTg      4.0
4      NUtIAX-ygn474tDg5nmesg  6LCZLGa09Qifn6rG7-DNrg      4.0
...                       ...                     ...      ...
97506  bJ5FtCtZX3ZZacz2_2PJjA  wMQkdK2aNMvq2xoojC98Mw      4.0
97507  bJ5FtCtZX3ZZacz2_2PJjA  SOsjW1JARmtHUFtpFlp8rw      4.0
97508  LHWtjTG7e1NzNPYUbUo-9w  rgeuy1qbw6Z8B6CSVANHIA      5.0
97509  7ziWZULyiZv2TesYNMFf4g  qQO7ErS_RAN4Vs1uX0L55Q      4.0
97510  MCzlzlOw7IGbRAKVjJBPtg  fcGexL5VH5G2Xw0tRj9uOQ      3.0

[97511 rows x 3 columns]>

# Step 3

<span style="color:black; font-weight:700;font-size:18px"> Διαβάζουμε τα έτοιμα από την εκφώνηση αρχεία train_data και test_data </span>

In [22]:
test_data = pd.read_csv('test_data.csv')
B_test = test_data.drop_duplicates(subset = ['business_id'])
test_data = test_data.drop('Unnamed: 0', axis=1)

train_data = pd.read_csv('train_data.csv')
train_data = train_data.drop('Unnamed: 0', axis=1)
train_data = train_data[train_data.business_id.isin(B_test['business_id'].values)]

<span style="color:black; font-weight:700;font-size:18px"> Δημιουργούμε μια συνάρτηση initialize που αρχικοποιεί το γράφημα μας. Σε κάθε κόμβο-χρήστη του βάζουμε και ένα κλειδί business_id που έχει για τιμή την βαθμολογία του χρήστη. Όπως λέει η εκφώνηση, όσοι κόμβοι-χρήστες περιέχονται στο train έχουν σταθερά τιμη r, ενώ όσοι κόμβοι-χρήστες περιέχονται στο test τους αρχικοποιούμε σε 0, μια τιμή που δεν μπορεί να υπάρξει ως βαθμολογία (1-5).</span>

In [23]:
def initialize(CC_max):
    for x in CC_max.nodes:
        # x is the user_id
        # get the reviews of user x
        reviews = train_data[train_data.user_id == x].values
        for u_b_r in reviews:
            # u_b_r[0]: the user_id (the x)
            # u_b_r[1]: the business_id
            # u_b_r[2]: the rating
            CC_max.nodes[x][u_b_r[1]] = u_b_r[2]
    
    for x in CC_max.nodes:
        # x is the user_id
        # get the reviews of user x
        reviews = test_data[test_data.user_id == x].values
        for u_b_r in reviews:
            # u_b_r[0]: the user_id (the x)
            # u_b_r[1]: the business_id
            # u_b_r[2]: the rating
            CC_max.nodes[x][u_b_r[1]] = 0

<span style="color:black; font-weight:700;font-size:18px"> Εδώ υλοποιείται το value propagation. Ξεκινάμε και παίρνουμε έναν-έναν τους κόμβους του test και για κάθε κλειδί-business_id κοιτάμε τις βαθμολογίες από τους γείτονες του και παίρνουμε το μέσο όρο τους. Φυσικά, πρώτα όσοι κόμβοι είναι κοντά στους κόμβους του train θα γίνει σε αυτούς πρώτα η διάχυση τιμών. Μα με τα iteration, και εκείνοι οι κόμβοι του test που θα είναι "πιο μακριά" θα γίνει διάχυση τιμών και σε αυτούς. Τερματίζουμε όταν φτάσουμε τα max iterations ή όταν γίνει ένα ολόκληρο iteration χωρίς μεγάλη αλλαγή.</span>

In [24]:
def value_propagation(CC_max, max_iter = 10):
    all_users = test_data.drop_duplicates(subset = ['user_id'])
    test_users = all_users['user_id'].values
    print("Max iters:",max_iter)
    iter_counter = 0
    user_counter = 0
    error_counter = 0
    review_counter = 0
    initialize(CC_max)
    while(True):
        # get the node that holds the user (the user is from test_data)
        user = CC_max.nodes[test_users[user_counter]]
        # get the list of the business that he has reviewed
        business_data = list(user.keys())
        # get the neighbors of the user
        neighbors = list(CC_max.neighbors(test_users[user_counter]))
        for bus in business_data:
            review_counter += 1
            # get current value of the stars. We initialized it at 0. (let's call it old)
            old_value = CC_max.nodes[test_users[user_counter]][bus]
            
            
            # here we hold the stars of the neighbors
            all_neighbor_stars = np.array([])
            
            # iterate through all friends, if they have reviewed the same business, append the stars
            for friend in neighbors:
                # here we hold the businesses of the neighbor
                neighbor_reviews = list(CC_max.nodes[friend].keys())
                if(bus in neighbor_reviews):
                    if(CC_max.nodes[friend][bus] == 0):
                        continue
                    all_neighbor_stars = np.append(all_neighbor_stars, CC_max.nodes[friend][bus])
            
            # calculate R
            if(len(all_neighbor_stars) != 0):
                R = np.sum(all_neighbor_stars) / len(all_neighbor_stars)
            else:
                R = 0
            
            # change the value
            CC_max.nodes[test_users[user_counter]][bus] = R
            # initialize back to empty array
            all_neighbor_stars = np.array([])
            
            if(not (old_value == 0 or R == 0)):
                if((abs(old_value-R) > 1/100000000)):
                    error_counter = 0
            error_counter += 1
                    
            
        # +1
        user_counter += 1
        
        if(user_counter == 100):
            error_counter = 0
            iter_counter += 1
            user_counter = 0
            review_counter = 0
        
        if(iter_counter == max_iter):
            print("Reached max iter")
            break
value_propagation(CC_max,70)

Max iters: 70
Reached max iter


<span style="color:black; font-weight:700;font-size:18px"> Μετά την διάχυση παίρνουμε όλα τα review στα εστιατόρια και δημιουργούμε ένα DataFrame με αυτές τις τριάδες, user_id business_id και stars. Μετά αφαιρούμε όσες τριάδες περιέχουν στην στήλη stars 0, πράγμα που σημαίνει ότι δεν έγινε διάχυση τιμής για αυτό το review.</span>

In [25]:
ll = []
result = pd.DataFrame()
test_users_list = test_data['user_id'].values
for x in CC_max.nodes:
        # x is the user_id
        # get the data of user x
        if(x not in test_users_list): continue
        user_dict = CC_max.nodes[x]
        local_df = pd.DataFrame(user_dict.items())
        local_df.columns = ['business_id','stars_pred']
        local_df['user_id'] = x
        local_df = local_df.reindex(columns=['user_id','business_id','stars_pred'])
        result = result.append(local_df)
merged_df = result.merge(test_data, how = 'inner', on = ['user_id','business_id'])

merged_df = merged_df[merged_df.stars_pred != 0]
merged_df.head

<bound method NDFrame.head of                     user_id             business_id  stars_pred  stars
0    FlXBpK_YZxLo27jcMdII1w  AlxnbBd2JNkozNvI0OvRLQ    5.000000    4.0
1    FlXBpK_YZxLo27jcMdII1w  AlxnbBd2JNkozNvI0OvRLQ    5.000000    3.0
2    FlXBpK_YZxLo27jcMdII1w  4_W5pstoN1166TGjjPOrMg    3.936042    4.0
3    FlXBpK_YZxLo27jcMdII1w  muCelHtTlX5PrJd8JKkf_w    3.400000    4.0
4    FlXBpK_YZxLo27jcMdII1w  AmI3LIUNwsi4023hOVGu3w    4.363636    2.0
..                      ...                     ...         ...    ...
792  Th-A3NlhCWBqbTzBZ9Tc2w  jkGQQ4_LgJx3hwPtCFkzbQ    3.000000    5.0
793  -2cKJFFNJ9XVyWBt62mWvA  L0zMk5SXnqR0JLEe0XY20g    4.122658    5.0
794  oNNL-ykTZx5S-xf66SOVjQ  _UOg5_pk9IhKee91eWrT4A    4.073849    5.0
795  oNNL-ykTZx5S-xf66SOVjQ  xQE1fuwKCXJEBcJgSSCNbw    3.946429    5.0
796  oNNL-ykTZx5S-xf66SOVjQ  dG-gZOWzn8iO1Rvv_fbXxA    3.802203    5.0

[648 rows x 4 columns]>

<span style="color:black; font-weight:700;font-size:18px"> To RMSE.</span>

In [26]:
merged_df['diff'] = merged_df['stars_pred'] - merged_df['stars']
merged_df['diff_sqr'] = merged_df['diff'].apply(lambda x: x**2)
sum_diff = merged_df.sum(axis=0)
rmse = np.sqrt(sum_diff[5]/len(merged_df['diff'].values))
print("RMSE:",rmse)

RMSE: 0.9414033539310246


<span style="color:black; font-weight:700;font-size:18px"> Διαβάζουμε τα fixed train/test set από την εκφώνηση. </span>

In [27]:
train_set = pd.read_csv("train_data.csv")
test_set = pd.read_csv("test_data.csv")

<span style="color:black; font-weight:700;font-size:18px"> Εδώ δημιουργούμε τον πίνακα με τις μέσες τιμές των user_id. Κάνουμε group_by και μέσα σε κάθε υπό-DataFrame του user_id βρίσκουμε το μέσο όρο της στήλης stars. </span>

In [28]:
user_groups = train_set.groupby('user_id')
user_scores_dic = {'user_id':[],'mean_star':[]}

for user_id, user_data in user_groups:
    user_scores_dic['user_id'].append(user_id)
    user_scores_dic['mean_star'].append(user_data['stars'].mean())

#user_scores = pd.DataFrame(user_scores_dic, columns = ['user_id','mean_star'])
user_scores = pd.DataFrame.from_dict(user_scores_dic)
user_scores.columns = ['user_id','mean_star']

In [29]:
user_scores.head()

,user_id,mean_star
0,-2cKJFFNJ9XVyWBt62mWvA,4.405405
1,-9OIms4jE1bdD1fz7AzAFA,3.967742
2,-A7wqNBMClbU6Y8NNLdMvw,4.096774
3,-G7Zkl1wIWBBmD0KRy_sCw,3.898551
4,-HYCAMf2ml717YD5Y9PKIg,3.617647


<span style="color:black; font-weight:700;font-size:18px"> Εδω υλοποιείται ο User Average (UA) αλγόριθμος. Για αρχή κάνουμε group_by σύμφωνα με τους users και διατρέχουμε τα dataframes του κάθε user. Με την χρήση του .loc και του user_id (από το iteration) βρίσκουμε στο datafrane user_scores την μέση τιμή του κάθε user και αφαιρούμε αυτήν την τιμή από όλες τια βαθμολογίες που έχει κάνει ο user στην στήλη 'stars'. Τετραγωνίζουμε την διαφορά και κρατάμε σε δύο παραμέτρους το άθροισμα τους και το πλήθος των χρηστών, πλήθος το όποιο ξέρουμε ήδη ούτως ή άλλως. </span>

In [30]:
user_groups_test = test_set.groupby('user_id')
error_sum = 0
count = 0

for user_id, user_data in user_groups_test:
    error_column_diff = user_data["stars"]-user_scores.loc[user_scores["user_id"] == user_id]['mean_star'].values[0]
    error_column_sqr = error_column_diff**2
    error_sum += error_column_sqr.sum()
    count += len(error_column_sqr)

In [31]:
start = time.time()
rmse = np.sqrt(error_sum/count)
print("RMSE of User Average:",rmse)
end = time.time()
print("Time:",end-start)

RMSE of User Average: 0.9012981678615513
Time: 0.0


<span style="color:black; font-weight:700;font-size:18px"> Εδώ δημιουργούμε τον πίνακα με τις μέσες τιμές των business_id. Κάνουμε group_by και μέσα σε κάθε υπό-DataFrame του business_id βρίσκουμε το μέσο όρο της στήλης stars. </span>

In [32]:
bus_groups = train_set.groupby('business_id')
bus_scores_dic = {'business_id':[],'mean_star':[]}

for business_id, bus_data in bus_groups:
    bus_scores_dic['business_id'].append(business_id)
    bus_scores_dic['mean_star'].append(bus_data['stars'].mean())

bus_scores = pd.DataFrame.from_dict(bus_scores_dic)
bus_scores.columns = ['business_id','mean_star']

In [33]:
bus_scores.head()

,business_id,mean_star
0,-0TffRSXXIlBYVbb5AwfTg,4.244000
1,-1B9pP_CrRBJYPICE5WbRA,3.761589
2,-4HMjEGQgduIMMTe0WPQBA,3.652174
3,-5Rah4ZvWsDu4oilUZxhtw,3.938596
4,-ATiAtTikuGuqvaW2O6tNA,3.471014


<span style="color:black; font-weight:700;font-size:18px"> Εδω υλοποιείται ο Business Average (BA) αλγόριθμος. Όπως πριν, για αρχή κάνουμε group_by σύμφωνα με τα business και διατρέχουμε τα dataframes του κάθε business. Με την χρήση του .loc και του business_id (από το iteration) βρίσκουμε στο datafrane bus_scores την μέση τιμή του κάθε business και αφαιρούμε αυτήν την τιμή από όλες τις βαθμολογίες που έχουν κάνει στην επιχείρηση στην στήλη 'stars'. Τετραγωνίζουμε την διαφορά και κρατάμε σε δύο παραμέτρους το άθροισμα τους και το πλήθος των επιχειρήσεων, πλήθος το όποιο ξέρουμε ήδη ούτως ή άλλως. </span>

In [34]:
bus_groups_test = test_set.groupby('business_id')
error_sum = 0
count = 0

for business_id, business_data in bus_groups_test:
    error_column_diff = business_data["stars"]-bus_scores.loc[bus_scores["business_id"] == business_id]['mean_star'].values[0]
    error_column_sqr = error_column_diff**2
    error_sum += error_column_sqr.sum()
    count += len(error_column_sqr)

In [35]:
start = time.time()
rmse = np.sqrt(error_sum/count)
print(rmse)
end = time.time()
print("Time:",end-start)

0.8698495174282991
Time: 0.0
